This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections


In [3]:
def gen_url(start_date,end_date,api_key,ticker=None):
    """
    Generates the url that can use to retrieve the information on a
    particular ticker for frankfurt stock exchange given start and end
    date.

    start_date: date from which the data is needed
    end_date: date till which data is needed
    ticker: the ticker for which the data is needed. defaults to AFX_X

    start_date and end_date format are 'YYYY-MM-DD'
    Warning: the function does not validate the dates

    """
    # locals
    _database_name = 'FSE'  # code of frankfurt stock exchange

    if not ticker:
        _ticker = 'AFX_X'  # ticker of carl zeiss
    else:
        _ticker = ticker

    begin_url =  'https://www.quandl.com/api/v3/datasets/'
    
    url = '{}{}/{}/data.json?start_date={}&end_date={}&api_key={}'.format(begin_url,_database_name,_ticker,start_date,end_date,api_key)

    return url

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

# getting url for one day (2020-02-19)
url = gen_url('2020-01-01','2020-01-08', API_KEY)

# creating response connection
resp = requests.get(url)

if resp:
    resp_json = resp.json()
else:
    print('FATAL: Can not Fetch url')
    resp_json = {}

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# inspecting data 
# type
print('type: {}'.format(type(resp_json)))

# list of keys
print('key list: {}'.format(list(resp_json.keys())))

# what is the value of the only key?
print('\n-----------------------')
print(resp_json['dataset_data'])
print('-----------------------')

# another dict. nested dict and that contains the actual data
print('\n-----------------------')
print('Keys in neseted dict')
print(resp_json['dataset_data'].keys())

# for the next activity we need column_names and data keys and their values

type: <type 'dict'>
key list: [u'dataset_data']

-----------------------
{u'column_names': [u'Date', u'Open', u'High', u'Low', u'Close', u'Change', u'Traded Volume', u'Turnover', u'Last Price of the Day', u'Daily Traded Units', u'Daily Turnover'], u'collapse': None, u'end_date': u'2020-01-08', u'transform': None, u'order': None, u'frequency': u'daily', u'limit': None, u'column_index': None, u'data': [[u'2020-01-08', 114.2, 116.8, 114.0, 116.4, None, 82884.0, 9602366.0, None, None, None], [u'2020-01-07', 114.4, 116.9, 114.3, 114.9, None, 98974.0, 11431904.0, None, None, None], [u'2020-01-06', 112.0, 114.3, 111.7, 114.0, None, 95697.0, 10844169.0, None, None, None], [u'2020-01-03', 112.8, 113.4, 112.0, 113.4, None, 82304.0, 9289987.0, None, None, None], [u'2020-01-02', 114.0, 114.9, 112.7, 113.8, None, 143464.0, 16291666.0, None, None, None]], u'start_date': u'2020-01-01'}
-----------------------

-----------------------
Keys in neseted dict
[u'column_names', u'collapse', u'end_date', u'

In [6]:
def process_raw_json(raw_json):

    # locals
    temp_dict = collections.defaultdict(list)

    raw_dict = raw_json['dataset_data']

    # columns 
    _columns = raw_dict['column_names']

    # stock data
    _data = raw_dict['data']

    # converting the data into processed format
    # what is done here is basically taking each column names 
    # from the columns and making them as a key and then taking 
    # the data and setting that as the value for the respective 
    # key
    for key, value in zip(_columns,zip(*_data)):
        temp_dict[key] = value

    return dict(temp_dict)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:
# 1 Data collection

url = gen_url('2017-01-01','2017-12-31',API_KEY)

response = requests.get(url)

response_json = response.json()

In [8]:
# 2 Convert JSON object to dictionary

data_dict = process_raw_json(response_json)

In [9]:
# 3 what is the highest and lowest opening prices for the stock in this period

print('Highest open price in this period was: {}'.format(str(max(data_dict['Open']))))

print('Lowest open price in this period was: {}'.format(str(min(data_dict['Open']))))

# the result is None which is not what we wanted. removing None and Doing the min operation again
# filtering all the non null values

filtered_open_prc_list = list(filter(lambda x: x!=None , data_dict['Open']))

print('Recitified Lowest open price in this period was: {}'.format(str(min(filtered_open_prc_list))))

Highest open price in this period was: 53.11
Lowest open price in this period was: None
Recitified Lowest open price in this period was: 34.0


In [10]:
# 4 What was the larget change in any one day
largest_change_list = [high-low for high,low in zip(data_dict['High'], data_dict['Low'])]
print('Largest change in any one day : {}'.format(str(max(largest_change_list))))

Largest change in any one day : 2.81


In [11]:
# 5 What was the larget change between any two days (based on closing price)
larget_change_2_day_list = [data_dict['Close'][i+1] - data_dict['Close'][i] for i in range(len(data_dict['Close'])-1)]

# Larget change can be negative or positive 
largest_change = 0
for change in larget_change_2_day_list:
    if abs(largest_change) < abs(change):
        largest_change = change
    else:
        pass

print('Largest change in any 2 days: {}'.format(str(largest_change)))

Largest change in any 2 days: 2.56


In [12]:
# 6 What wsa the average daily trading volume during this year
total_volume_sum = sum(data_dict['Traded Volume'])
size_of_traded_vol_list = len(data_dict['Traded Volume'])
average = total_volume_sum/size_of_traded_vol_list

print('Average daily traded volume during this year(rounded to nearst 2 decimal): {}'.format(str(round(average,2))))

Average daily traded volume during this year(rounded to nearst 2 decimal): 89124.34


In [13]:
# 7 What was the median trading volume during this year
# logic is to arrange the number in ascending order
def median(list_val):
    sorted_list = sorted(list_val)
    if len(sorted_list)%2 == 0:   # when length of list is even
        index = len(sorted_list)/2
        median = (sorted_list[index]+sorted_list[index+1])/2.
    else:
        index = len(sorted_list)/2
        median = sorted_list[index]
    
    return median

In [14]:
median_volume = median(data_dict['Traded Volume'])

print('Median trading volume: {}'.format(median_volume))

Median trading volume: 76286.0
